# [Create Agent with Azure AI Search](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-ai-search)
Azure AI Search is an enterprise search system for high-performance applications. It integrates with Azure OpenAI Service and Azure Machine Learning, offering advanced search technologies like vector search and full-text search. Ideal for knowledge base insights, information discovery, and automation.<br/><br/>
**IMPORTANT**: in order to create the index leveraging the "Import and Vectorize" wizard, the `Storage Blob Data Contributor` role on the Storage Account is needed for the Azure AI Managed Identity.

## Create a connection to Azure AI Search using CLI

1. First of all, let's check which connections we have, associated to our project mmai-hub04-prj01-fvye:
   ```az ml connection list --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye```
3. Now, create a new yaml file with the configuration for Azure AI Search, using key-based or key-less (as in this case) authentication. **Please note that the "metadata" section must be filled as shown, including the ResourceId that must contain the connection name reporten on line 1**:
```
name: mmai-hub04-fvye-connection-AISearch
type: azure_ai_search
endpoint: https://mmai-hub04-ai-search-fvye.search.windows.net/
is_shared: true
metadata:
  ApiType: Azure
  ResourceId: /subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.Search/searchServices/mmai-hub04-fvye-connection-AISearch
  type: azure_ai_search
  ```

3. Run the command `az ml connection create --file aisearchconnection.yml --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye`

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        print(f">>>>>>> AI SEARCH FOUND: {conn}")
        conn_id = conn.id
    else:
        print(conn)

{
 "name": "mmai-hub04-fvye-connection-AIServices_aoai",
 "id": "/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AIServices_aoai",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_OPEN_AI",
 "endpoint_url": "https://mmai-hub04-ai-servicesfvye.openai.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "mmai-hub04-fvye-connection-AIServices",
 "id": "/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AIServices",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_AI_SERVICES",
 "endpoint_url": "https://mmai-hub04-ai-servicesfvye.cognitiveservices.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "grounding_with_bing_search_connection

In [4]:
conn_id

'/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch'

In [5]:
# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)
print(f"ai_search.definitions: {ai_search.definitions}")
print(f"ai_search.resources: {ai_search.resources}")

ai_search.definitions: [{'type': 'azure_ai_search'}]
ai_search.resources: {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}


In [6]:
# Check if we already have any agents defined
project_client.agents.list_agents()

{'object': 'list', 'data': [], 'first_id': None, 'last_id': None, 'has_more': False}

In [7]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"},
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'object': 'assistant', 'created_at': 1735902598, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}, 'metadata': {}, 'response_format': 'auto'}>

In [8]:
# Check again if the agent created above is now present here
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'object': 'assistant', 'created_at': 1735902598, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}, 'metadata': {}, 'response_format': 'auto'}], 'first_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'last_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'has_more': False}

In [9]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

Created thread: {'id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'object': 'thread', 'created_at': 1735902599, 'metadata': {}, 'tool_resources': {}}


In [10]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="What kind of keys does the MS Surface Book keyboard include? Do **NOT** use your own internal knowledge." # "How does wikipedia explain Euler's Identity?"
)
print(f"Created message: {message}")

Created message: {'id': 'msg_GbAMhGEs6YPn4jRPB1hbCpin', 'object': 'thread.message', 'created_at': 1735902599, 'assistant_id': None, 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What kind of keys does the the MS Surface Book keyboard include? Do **NOT** use your own internal knowledge.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [11]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED. Run: {'id': 'run_z5qTBFOEWQ44O3i5VkSSo7Og', 'object': 'thread.run', 'created_at': 1735902600, 'assistant_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'status': 'completed', 'started_at': 1735902600, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1735902607, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 1204, 'completion_tokens': 100, 'total_tokens': 1304}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 15.6 ms
Wall time: 7.7 s


In [12]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
messages

{'object': 'list', 'data': [{'id': 'msg_u0UGkwOehF7i4u6BzoeN7Acz', 'object': 'thread.message', 'created_at': 1735902606, 'assistant_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'run_id': 'run_z5qTBFOEWQ44O3i5VkSSo7Og', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The Surface Book keyboard has a standard layout and includes backlit keys, special shortcut keys, and function keys. The keys on the top row double as function keys, which can be used by holding down the Fn key while pressing a top-row key. The keyboard also includes specific shortcuts for tasks such as adjusting backlighting, playing or pausing media, muting audio, and adjusting the volume【7:0†source】.', 'annotations': [{'type': 'file_citation', 'text': '【7:0†source】', 'start_index': 393, 'end_index': 405, 'file_citation': {'file_id': 'surface-book-user-guide-EN.pdf'}}, {'type': 'file_citation', 'text': '【7:0†source】', 'start_index': 393, 'end_index': 405, 'file

In [13]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: The Surface Book keyboard has a standard layout and includes backlit keys, special shortcut keys, and function keys. The keys on the top row double as function keys, which can be used by holding down the Fn key while pressing a top-row key. The keyboard also includes specific shortcuts for tasks such as adjusting backlighting, playing or pausing media, muting audio, and adjusting the volume【7:0†source】.


In [14]:
# Print citations from the messages
for annotation in messages.file_citation_annotations:
    print(annotation["text"], annotation["file_citation"]["file_id"])

【7:0†source】 surface-book-user-guide-EN.pdf
【7:0†source】 surface-book-user-guide-EN.pdf


In [15]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

Last run step detail: [{'id': 'step_RXCukcyWXUrWPHeGl59FCwmV', 'object': 'thread.run.step', 'created_at': 1735902606, 'run_id': 'run_z5qTBFOEWQ44O3i5VkSSo7Og', 'assistant_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735902607, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_u0UGkwOehF7i4u6BzoeN7Acz'}}, 'usage': {'prompt_tokens': 1204, 'completion_tokens': 88, 'total_tokens': 1292}}, {'id': 'step_rhOzg9gYt8ISIKtCHI9WYb5I', 'object': 'thread.run.step', 'created_at': 1735902604, 'run_id': 'run_z5qTBFOEWQ44O3i5VkSSo7Og', 'assistant_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735902606, 'expires_at': None, 'failed_at': None, 'last_error': None,

In [16]:
for rs in run_steps["data"]:
    print(rs, '\n')

{'id': 'step_RXCukcyWXUrWPHeGl59FCwmV', 'object': 'thread.run.step', 'created_at': 1735902606, 'run_id': 'run_z5qTBFOEWQ44O3i5VkSSo7Og', 'assistant_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735902607, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_u0UGkwOehF7i4u6BzoeN7Acz'}}, 'usage': {'prompt_tokens': 1204, 'completion_tokens': 88, 'total_tokens': 1292}} 

{'id': 'step_rhOzg9gYt8ISIKtCHI9WYb5I', 'object': 'thread.run.step', 'created_at': 1735902604, 'run_id': 'run_z5qTBFOEWQ44O3i5VkSSo7Og', 'assistant_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'thread_id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1735902606, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'typ

# START teardown

In [17]:
project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'object': 'assistant', 'created_at': 1735902598, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}, 'metadata': {}, 'response_format': 'auto'}], 'first_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'last_id': 'asst_PyHOBnnF7bxu6UyQNzt3e0Jf', 'has_more': False}

In [18]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'object': 'thread', 'created_at': 1735902599, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_WbeQkWTfZovU5OPTlQb3QWK2', 'object': 'thread.deleted', 'deleted': True}

In [19]:
for f in project_client.agents.list_files()['data']:
    print(f"deleting file: {f}...")
    project_client.agents.delete_file(f.id)

In [20]:
for v in project_client.agents.list_vector_stores()['data']:
    print(f"deleting vectore store: {v}...")
    project_client.agents.delete_vector_store(v.id)

In [21]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-assistant (asst_PyHOBnnF7bxu6UyQNzt3e0Jf)...


# HIC SUNT LEONES